In [1]:
from os.path import abspath
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
import os
import sys
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
os.environ['PYSPARK_DRIVER_PYTHON_OPTS']= "notebook"
os.environ['PYSPARK_DRIVER_PYTHON'] = sys.executable
os.environ['PYSPARK_PYTHON'] = sys.executable

spark = SparkSession \
    .builder \
    .appName("renovation") \
    .getOrCreate()

# set this parameter for date issue before 1582 (dpe database)
spark.conf.set("spark.sql.legacy.timeParserPolicy","LEGACY")

In [3]:
file_location = "../../data/dpe-v2-logements-existants.csv"
file_type = "csv"
name = "dpe_2021"

# CSV options
infer_schema = "true"
first_row_is_header = "true"
delimiter = ","

# The applied options are for CSV files. For other file types, these will be ignored.
df = spark.read.format(file_type) \
    .option("inferSchema", infer_schema) \
    .option("header", first_row_is_header) \
    .option("sep", delimiter) \
    .load(file_location)
if name == "weather":
    new_column_name_list= [name.replace(',','') for name in df.columns]
    df = df.toDF(*new_column_name_list)
# df.write.mode('overwrite')\
#     .format("parquet") \
#     .saveAsTable(f"Datalake.{name}")

AnalysisException: [SCHEMA_NOT_FOUND] The schema `datalake` cannot be found. Verify the spelling and correctness of the schema and catalog.
If you did not qualify the name with a catalog, verify the current_schema() output, or qualify the name with the correct catalog.
To tolerate the error on drop use DROP SCHEMA IF EXISTS.

In [9]:
df.show()
# [print(column) for column in df.columns]

+--------------------+------------------+----------------------+--------------------------+--------------------+--------------------+---------------------+-----------+-----------------------+----------------------------+-----------------------+------------------------------+-------------------------+----------------+-------------+-------------+------------------+--------------------+---------------------------+-------------------------------+--------------------+--------------------+------------------+----------------------+----------------------+--------------------------+--------------------------+--------------------------+-----------------------+------------------+-------------------------------+----------------+----------------+--------------------+------------------+----------------+-------------+----------------+--------------------+-----------------+---------+--------------------+---------------------------------+---------------------------------+------------------+------------

In [14]:
dpe_2021 = (
    df.select(
        "N°DPE",
        "Date_établissement_DPE",
        "Modèle_DPE",
        "N°_DPE_remplacé",
        "Version_DPE",
        "N°_DPE_immeuble_associé",
        "Méthode_application_DPE",
        "Etiquette_GES",
        "Etiquette_DPE",
        "Année_construction",
        "Type_bâtiment",
        "Type_installation_chauffage",
        "Type_installation_ECS_(général)",
        "Période_construction",
        "Nombre_appartement",
        "Nombre_niveau_immeuble",
        "Nombre_niveau_logement",
        "Surface_habitable_immeuble",
        "Surface_habitable_logement",
        "Position_logement_dans_immeuble",
        "Classe_altitude",
        "Code_INSEE_(BAN)",
        "Adresse_(BAN)",
        "Code_postal_(BAN)",
        "N°_étage_appartement",
        "Conso_5_usages_é_finale",
        "Conso_5_usages/m²_é_finale",
        "Emission_GES_5_usages",
        "Emission_GES_5_usages_par_m²",
        "Type_installation_ECS",
        "Description_installation_ECS",
        "Type_énergie_générateur_ECS_n°1",
        "Type_générateur_ECS_n°1",
        "Type_énergie_principale_chauffage",
        "Type_installation_chauffage_n°1",
        "Type_émetteur_installation_chauffage_n°1",
        "Description_générateur_chauffage_n°1_installation_n°1",
        "Type_énergie_générateur_n°1_installation_n°1",
        "Type_générateur_n°1_installation_n°1",
        "Ubat_W/m²_K",
        "Qualité_isolation_plancher_haut_toit_terrase",
        "Qualité_isolation_plancher_haut_comble_aménagé",
        "Qualité_isolation_plancher_haut_comble_perdu",
        "Qualité_isolation_enveloppe",
        "Qualité_isolation_menuiseries",
        "Qualité_isolation_murs",
        "Qualité_isolation_plancher_bas"
    )
)
dpe_2021.show()

+--------------------+----------------------+--------------------+--------------------+-----------+-----------------------+-----------------------+-------------+-------------+------------------+--------------------+---------------------------+-------------------------------+--------------------+------------------+----------------------+----------------------+--------------------------+--------------------------+-------------------------------+----------------+----------------+--------------------+-----------------+--------------------+-----------------------+--------------------------+---------------------+----------------------------+---------------------+----------------------------+-------------------------------+-----------------------+---------------------------------+-------------------------------+----------------------------------------+-----------------------------------------------------+--------------------------------------------+------------------------------------+-------

In [ ]:
dpe_2021_